In [1]:
import numpy as np
from scipy.io import loadmat
import tensorflow as tf
import glob
import time
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
def plot_eeg(data):
    plt.figure(figsize=(10,20))
    for i in range(0,16):
        plt.subplot(8,2,i+1)
        plt.plot(data[:,i])
    
    plt.savefig('foo.pdf', bbox_inches='tight')

In [3]:

def mat2tfr(p_file):
    #converting mat file as numpy
    mat = loadmat(p_file)
    data = mat['dataStruct']['data'][0][0]


    #getting the filename and retrieving the patient, segement and label data
    pat, seg, label = p_file.split('/')[-1].split('.')[0].split("_")

    dstfolder = "./datasets/train_1_tfr/"
    dstfilename = dstfolder +pat+"_"+seg+"_"+label+".tfr"
    print("Converting " + p_file + " ----> " + dstfilename)

    #creating the TensorFlow Records 
    tfrwriter = tf.python_io.TFRecordWriter(dstfilename)
    
    #plot_eeg(data)
    

    
    #reshape and transform to list due to TFRecords
    data = np.reshape(data, -1).tolist()

    #filling protobuff
    protobuf = tf.train.Example(
        features=tf.train.Features(feature={
            'label':  tf.train.Feature(int64_list = tf.train.Int64List(value= [int(label)])),
            'data' :  tf.train.Feature(float_list = tf.train.FloatList(value= data))
    }))

    tfrwriter.write(protobuf.SerializeToString())
    tfrwriter.close()
    
        

def dataset2tfr(folder):
    filenames = glob.glob(folder)
    print("Converting #%d files."%len(filenames))
    
    for files in filenames:
        t = time.time()
        
        mat2tfr(files)

        elapsed = time.time() - t
        print("elapsed: %.3fs"%elapsed)
        
        
dataset2tfr("./datasets/train_1/1_1_*.mat")
        

Converting #2 files.
Converting ./datasets/train_1/1_1_0.mat ----> ./datasets/train_1_tfr/1_1_0.tfr
elapsed: 2.734s
Converting ./datasets/train_1/1_1_1.mat ----> ./datasets/train_1_tfr/1_1_1.tfr
elapsed: 2.590s


In [4]:
def tfr_reader(folder):
    filenames = glob.glob(folder)
    print("Loading #%d files."%len(filenames))
    
    
    #generate file queue
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=2, shuffle=True) 
    
    #open reader and read queue
    reader = tf.TFRecordReader()
    _, protobuf = reader.read(filename_queue)
    
    
    features = tf.parse_single_example(
      protobuf,
      # Defaults are not specified since both keys are required.
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'data': tf.FixedLenFeature([], tf.float32)
      })
    
    
    print(features)

#     Convert label from a scalar uint8 tensor to an int32 scalar.
#     label = tf.cast(features['label'], tf.int64)
#     print(label)
#     data = tf.cast(features['data'], tf.float32)
#     print(data)
    

    
    
    
    
tfr_reader("./datasets/train_1_tfr/*.tfr")

Loading #2 files.
{'data': <tf.Tensor 'ParseSingleExample/Squeeze_data:0' shape=() dtype=float32>, 'label': <tf.Tensor 'ParseSingleExample/Squeeze_label:0' shape=() dtype=int64>}


## Questions


    - Are all datasets 240000x16?
    - Do the 16 sensors in the same brain locations?
    - 